#Analysis of Ad-Hoc Communications Network
The user can supply the overall size of desired coverage footprint andthen we will determine the following:

- Given an overall desired coverage footprint and a sequence of n communications
towers, what is the resulting resolved coverage?
- What is the total area of coverage relative to the desired total coverage area of the
original footprint? That is, are there any gaps in coverage?
- On average, how many communications towers are required before full coverage is
obtained?

Inputs w and h represent width and height of the desired region map to analyze, respectively 

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def create_map(w,h):
    """
    Create a blank region area based on user input w and h, and plot the empty map
    :param w: width of the desired area 
    :param h: height of desired area
    :return: numpy area of 0's which will represent the blank map area
    """
    assert isinstance(w, int)
    assert isinstance(h, int)
    assert w>0
    assert h>0
    
    xvalues = np.array(range(w));
    yvalues = np.array(range(h));
    
    xx,yy = np.meshgrid(xvalues, yvalues)
    
    blank = np.zeros(h,w)
    plt.plot(xx,yy, marker='.', color='k', linestyle='none')
    return x

Input n represents the numbers of towers

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches


def add_tower(n,w,h):
    """
    Adds tower, and yreturns so that each step can be shown in a visualization
    :param n: number of towers
    :param w: width of region map
    :param h: height of region map
    :return: updated map with one tower added
    """
    import random
    import itertools
    assert isinstance(n, int)
    assert isinstance(w,int)
    assert isinstance(h,int)
    assert n>0 and w>0 and h>0
    
    # determines which corner to start at
    s_cornerw = random.randint(0, w+1)
    s_cornerh = random.randint(0, h+1)
        
    # determines tower random width and height of rectangle (minimum 1)
    tow_width = random.randint(1, w+1)
    tow_height = random.randint(1, h+1)
    
    # update numpy array
    
    

    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines 
import matplotlib.patches as patches

def plot_tower(n, s_cornerw, s_cornerh, tow_width, tow_height):
    
    # update to map, plots 4 lines for the rectangle
    plt.plot([s_cornerw, s_cornerw+tow_width], [s_cornerh, s_cornerh])
    plt.plot([s_cornerw+tow_width, s_cornerw+tow_width], [s_cornerh, s_cornerh+tow_height])
    plt.plot([s_cornerw+tow_width, s_cornerw], [s_cornerh+tow_height, s_cornerh+tow_height])
    plt.plot([s_cornerw, s_cornerw], [s_cornerh+tow_height, s_cornerh])
    
    # better way to plot 
    patches.rectangle(s_cornerw,s_cornerh, tow_width, tow_height)

In [4]:
def main(n, w, h):
    """
    Combines functions and get user input
    :return: finished map, and every updated tower
    """
    
    x = create_map(w,h)
    
    
    